In [21]:
import numpy as np
import matplotlib.pyplot as plt
import psycopg2

from datetime import datetime, timezone

### SQL settings

In [22]:
DBNAME = "fesa_test"
HOST = "140.181.85.66"
PORT = "54321"
USER = "fesa_tester"
PASSWORD = "save_bpm"

In [23]:
try:
    dbcon = psycopg2.connect("dbname="+DBNAME + " user=" + USER + " host=" + HOST + " port=" + PORT + " password=" + PASSWORD)
except Exception as e:
    print("Unable to connect to database")
    print(e)

crsr = dbcon.cursor()

### Sequence start

In [24]:
def stamp2time(stamp):
    ts = stamp / 1e9
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

In [25]:
def fetch_sequenceStartStamp(crsr=crsr):
    stamps = crsr.execute("SELECT sequenceStartStamp FROM gs01dx_acquisition;")
    
    return crsr.fetchall()

In [26]:
stamps = fetch_sequenceStartStamp()
first_record = stamps[0][0]
last_record = stamps[-1][0]

stamp2time(last_record)

'1970-01-01 00:00:00.000000'

# Read BPM

In [27]:
date = "2023-10-25"
# time = "08:21:35.0"
time = "12:22:19.0"

In [28]:
time_datetime = datetime.strptime(" ".join([date,time]), '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=timezone.utc)
stamp_given = int(time_datetime.timestamp() * 1e9)

stamps_arr = np.array(stamps).squeeze(axis=1)
idx_closest_stamp = np.abs(stamps_arr - stamp_given).argmin()

In [29]:
idx_closest_stamp

0

###### selected cycle

In [30]:
display_stamp = stamps_arr[idx_closest_stamp]

stamp2time(display_stamp)

'2023-10-25 13:49:31.208560'

# fetch beamPosition for sequenceStartStamp

In [31]:
def bpmReadings_demasked(no_bpm: int, sequenceStartStamp: int, crsr=crsr):
    statement = """
    SELECT
        processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp
    FROM
        gs{:02d}dx_acquisition
    WHERE
        sequenceStartStamp = %s
    ORDER BY
        processStartStamp
;
    """.format(no_bpm)
    
    crsr.execute(statement, [int(sequenceStartStamp),])
    data = crsr.fetchall()
    
    assert len(data) > 0, "nothing selected"
    
    # flatten over all beamprocesses in this pattern
    x_raw, y_raw, time = list(), list(), list()
    for process in data:
        timepoints = len(process[2])
        time += process[2]
        
        assert 4*timepoints == len(process[1]), "mask assumption wrong"
        
        x = process[1][0:timepoints]
        y = process[1][2*timepoints:3*timepoints]
        
        x_raw += x
        y_raw += y
        
    return np.array(x_raw), np.array(y_raw), np.array(time)

In [32]:
def findKickTime(x_raw, y_raw, time, harmonic_number: int):
    ######## implement some criterion to locate kick
    
    ##### fake 
    l = len(time)
    kick_times = time[l//2:l//2+harmonic_number]

    # calculate revolution period
    Trev = time[l//2 + harmonic_number] - time[l//2]
    
    return kick_times, Trev

### fetch

In [33]:
crsr.execute("""
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs01dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs02dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs03dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs04dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs05dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs06dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs07dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs08dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs09dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs10dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs11dx_acquisition WHERE sequenceStartStamp = %s
    UNION
    SELECT processIndex, beamPosition, beamPosition_dim3_labels, processStartStamp FROM gs12dx_acquisition WHERE sequenceStartStamp = %s
    ;
    """, [int(display_stamp) for _ in range(12)])
data = crsr.fetchall()

assert len(data) == 12, "missing some BPMs"

In [34]:
harmonic_number = 4

In [35]:
trajectory = np.empty((12,3,1024))  # bpm starting from 5, x/y/time, window length

for i, no_bpm in enumerate([5,6,7,8,9,10,11,12,1,2,3,4]):
    x_raw, y_raw, time = bpmReadings_demasked(no_bpm, display_stamp)
        
    if no_bpm == 5:
        kick_times, Trev = findKickTime(x_raw, y_raw, time, harmonic_number)
        print("found kick_times")
    
    print("\n", no_bpm)
    print("timestamp distance", time[1001] - time[1000])
#     print(stamp2time(time[-1]))
    
    for no_bunch, kickTime in enumerate(kick_times):
        retarded_kickTime = kickTime + i * Trev/12
        assert retarded_kickTime >= kickTime
        
        kick_idx = np.abs(time - retarded_kickTime).argmin()
        
        print("minimum distance to kickTime", np.abs(time - kickTime).min())
        print("minimum distance to retarded_kickTime", np.abs(time - retarded_kickTime).min())
        print("diff", abs(time[kick_idx] - retarded_kickTime))
        
#         assert abs(time[kick_idx] - retarded_kickTime) < Trev/12, "kick_idx far away from kickTime"
#         assert np.abs(time - retarded_kickTime).min() < Trev/12, "kick_idx far away from kickTime"
        
        traj_mask = np.zeros(time.shape, dtype=bool)
        traj_mask[kick_idx::harmonic_number] = True
        
        trajectory[i,0,:] = x_raw[traj_mask][:trajectory.shape[2]]
        trajectory[i,1,:] = y_raw[traj_mask][:trajectory.shape[2]]
        trajectory[i,2,:] = time[traj_mask][:trajectory.shape[2]]

found kick_times

 5
timestamp distance 1104
minimum distance to kickTime 0
minimum distance to retarded_kickTime 0.0
diff 0.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 0.0
diff 0.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 0.0
diff 0.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 0.0
diff 0.0

 6
timestamp distance 1104
minimum distance to kickTime 0
minimum distance to retarded_kickTime 256.0
diff 256.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 256.0
diff 256.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 256.0
diff 256.0
minimum distance to kickTime 0
minimum distance to retarded_kickTime 256.0
diff 256.0

 7
timestamp distance 1104
minimum distance to kickTime 4
minimum distance to retarded_kickTime 256.0
diff 256.0
minimum distance to kickTime 4
minimum distance to retarded_kickTime 512.0
diff 512.0
minimum distance to kickTime 4
minimum distance to reta

# debug

In [36]:
dbcon.reset()

In [37]:
Trev/12

368.6666666666667

In [38]:
trajectory[:,[0,1],:]

array([[[ 8.0034165e-03,  6.3008370e-03,  2.5536683e-03, ...,
          2.7166517e-03, -7.1852556e-03,  6.5168296e-04],
        [ 1.5481304e-03, -6.3534360e-03,  1.6067014e-03, ...,
         -2.6701610e-03,  2.0563684e-03, -6.3540270e-03]],

       [[-3.4052308e-03, -3.6918342e-03, -4.4260197e-03, ...,
          2.1275901e-03, -5.5359000e-03, -6.7073847e-03],
        [-4.3771323e-03,  6.9663810e-03,  1.5748415e-02, ...,
         -8.4155480e-03,  6.1064917e-03, -1.2882301e-02]],

       [[ 2.3213793e-02,  5.2975020e-03,  1.3360818e-02, ...,
          2.1465212e-02,  1.1551910e-02,  1.4972624e-02],
        [ 9.9371366e-02,  9.9593244e-02,  9.8889900e-02, ...,
          9.9165164e-02,  9.7618920e-02,  9.8587330e-02]],

       ...,

       [[-1.1796489e-02, -2.4019901e-03, -8.6014280e-03, ...,
          7.7376590e-03,  1.3008982e-02, -9.7534690e-03],
        [ 2.8071650e-02,  3.0641474e-02,  3.4621876e-02, ...,
          3.8340690e-02,  3.2033470e-02,  3.6153484e-02]],

       [[ 1.8065957

In [41]:
x_raw.shape

(10000,)